In [1]:
sc

sqlc = new org.apache.spark.sql.SQLContext(sc)

In [ ]:
schemaPeople = sqlContext.createDataFrame(people)
schemaPeople.registerTempTable("people")

In [2]:
l = [1,2,4,5,6,8]

In [3]:
z = sc.parallelize(l)


In [4]:
print type(z)

<class 'pyspark.rdd.RDD'>


In [5]:
z.map(lambda x : x**3).take(3)

[1, 8, 64]

In [6]:
z.map(lambda x : x**2).take(3)

[1, 4, 16]

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=5, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda (v, p): (v - p) * (v - p)).sum() / float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression tree model:')
print(model.toDebugString())

# Save and load model
model.save(sc, "myModelPath")
sameModel = DecisionTreeModel.load(sc, "myModelPath")